In [ ]:
pip install telebot


In [ ]:
pip install opencv-python

In [ ]:
pip install rembg

In [23]:
import os
import cv2
import numpy as np
import tensorflow as tf
from telebot import *
# from telebot import types

bot = telebot.TeleBot("Token")
model = tf.keras.models.load_model('/content/drive/MyDrive/17_Flowers.h5')
model_Akhund_Human = tf.keras.models.load_model('/content/drive/MyDrive/Data/Akhund_and_Human.h5')

flowers_name = ['bluebell', 'buttercup', 'coltsfoot', 'cowslip', 'crocus', 'daffodil','daisy', 'dandelion', 'fritillary', 'iris', 'lilyvalley', 'pansy', 'snowdrop', 'sunflower', 'tigerlily', 'tulip', 'windflower']


@bot.message_handler(commands=["start"])
def welcom(message):
    bot.reply_to(message, f"✋welcom  to {message.chat.first_name} boot AI ")
@bot.message_handler(commands=["help"])
def help(message):
	bot.send_message(message.chat.id,"/gray - Imaeg gray 🌅 \n/sketch - Photo Sketch 📸\n/flowers - photo Flower detection🌹🌿\n/akhund - distinguish mullah and human👳🏻‍♂️👨🏻")

@bot.message_handler(commands=['gray'])
def send_pencil_sketch(message):
    global bot_state
    bot.send_message(message.chat.id, "Send me an image 🌅")
    bot_state = 'gray'

@bot.message_handler(commands=['sketch'])
def send_pencil_sketch(message):
    global bot_state
    bot.send_message(message.chat.id, "Send me an image 🌅")
    bot_state = 'Sketch'

@bot.message_handler(commands=['flowers'])
def send_photo(message):
    msg = bot.send_message(message.chat.id,"Send me an image flower🌹")
    bot.register_next_step_handler(msg,photo)

@bot.message_handler(commands=['akhund'])
def send_photo_akhunb(message):
    msg = bot.send_message(message.chat.id,"Send me an image 👳🏻‍♂️👨🏻")
    bot.register_next_step_handler(msg,recognizer)

def save_image(file_info, image_name=None):
    downloaded_file = bot.download_file(file_info.file_path)
    if image_name:
        image_path = os.path.join('photos', image_name + '.jpg')
    else:
        image_path = os.path.join(file_info.file_path)

    with open(image_path, 'wb') as new_file:
        new_file.write(downloaded_file)
    return image_path

@bot.message_handler(content_types=['photo'])
def send_photo(message):
    global bot_state, content_image_path
    file_info = bot.get_file(message.photo[-1].file_id)
    if bot_state == 'gray':
        image_path = save_image(file_info)
        image_result = image_gray(image_path)
    elif bot_state == 'Sketch':
        image_path = save_image(file_info)
        image_result = Sketch(image_path)

    image_path = os.path.join( file_info.file_path)
    cv2.imwrite(image_path, image_result)

    photo = open(image_path, "rb")
    bot.send_photo(message.chat.id, photo)

def image_gray(image_path):
    image = cv2.imread(image_path)
    image_result = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image_result
def dodge(x, y):
    return cv2.divide(x, 255 - y, scale=256)

def burn(image, mask):
    return 255 - cv2.divide(255 - image, 255 - mask, scale=256)

def Sketch(image_path):
    image = cv2.imread(image_path)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_gray_inv = 255 - image_gray
    image_gray_inv_blur = cv2.GaussianBlur(image_gray_inv, (21, 21), sigmaX=0, sigmaY=0)
    image_dodged = dodge(image_gray, image_gray_inv_blur)
    image_result = burn(image_dodged, image_gray_inv_blur)
    return image_result



@bot.message_handler(content_types = ["flowers"])
def photo(message):
    fileID = message.photo[-1].file_id
    file_info = bot.get_file(fileID)
    downloaded_file = bot.download_file(file_info.file_path)

    with open("Flowers.jpg", 'wb') as new_file:
        new_file.write(downloaded_file)

    img = cv2.imread("Flowers.jpg")
    image = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(image ,(224,224))
    img = img / 255
    img = img.reshape(1,224,224,3)
    result = np.argmax(model.predict(img))
    print(result)
    print(flowers_name[result])

    bot.send_message(message.chat.id,f' I feel your name is flowers {flowers_name[result]} ☘️🪴 .')


def recognizer(message):
    fileID = message.photo[-1].file_id
    file_info = bot.get_file(fileID)
    downloaded_file = bot.download_file(file_info.file_path)
    width = height = 224


    with open("image.jpg", 'wb') as new_file:
        new_file.write(downloaded_file)

    img = cv2.imread("image.jpg")
    image = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(image ,(width,height))
    img = img / 255
    img = img.reshape(1,width,height,3)

    result = np.argmax(model_Akhund_Human.predict(img))
    print(result)

    if result == 0:
        bot.send_message(message.chat.id,'از ما نیست👳🏻‍♂️')

    elif result == 1:
        bot.send_message(message.chat.id,'از ماست 👨🏻👩🏻')



bot.infinity_polling()



1/1 [==============================] - 1s 1s/step
1
1/1 [==============================] - 0s 28ms/step
0


2023-10-30 19:47:58,624 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2023-10-30 19:47:58,628 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
